In [1]:
# Import the 'random' module for generating random numbers and performing random operations.
import random

In [2]:
# Initialize an empty 3x3 Tic-Tac-Toe board as a list of lists with blank spaces
board = [[' ' for _ in range(3)] for _ in range(3)]

# Define the AI difficulty levels as constants
EASY = 'easy'
HARD = 'hard'

# Initialize game statistics, keeping track of player wins, AI wins, and draws
game_stats = {'player_wins': 0, 'ai_wins': 0, 'draws': 0}

# Initialize a move history list to keep track of moves made during the game
move_history = []

In [3]:
# Function to reset the game state by clearing the Tic-Tac-Toe board
def reset_game():
    # Loop through each row and column on the board
    for i in range(3):
        for j in range(3):
            # Set each cell to an empty space to clear the board
            board[i][j] = ' '

In [4]:
# Function to display the current state of the Tic-Tac-Toe board
def display_board(board):
    # Loop through each row in the board
    for row in board:
        # Print the cells in the current row, separated by '|'
        print('|'.join(row))
        # Print a horizontal line to separate rows
        print('-' * 5)

In [5]:
# Function to check if a player has won the Tic-Tac-Toe game
def check_winner(board, player):
    # Check rows and columns for a win
    for i in range(3):
        if all(board[i][j] == player for j in range(3)) or \
           all(board[j][i] == player for j in range(3)):
            return True

    # Check diagonals for a win
    if all(board[i][i] == player for i in range(3)) or \
       all(board[i][2 - i] == player for i in range(3)):
        return True

    # If no win condition is met, return False
    return False

In [6]:
# Function to check if the game has ended in a draw
def check_draw(board):
    # The game is a draw if there are no empty cells left on the board
    return all(board[i][j] != ' ' for i in range(3) for j in range(3))

# Function to make a random move on the board
def random_move(board):
    # Find all empty cells on the board
    empty_cells = [(i, j) for i in range(3) for j in range(3) if board[i][j] == ' ']

    # If there are empty cells, choose a random one; otherwise, return None
    return random.choice(empty_cells) if empty_cells else None

In [7]:
# Minimax function with Alpha-Beta Pruning to determine the best move for the AI ('O')
def minimax(board, depth, is_maximizing, alpha, beta):
    # Base cases for terminal states (win, loss, draw)
    if check_winner(board, 'X'):
        return -1
    if check_winner(board, 'O'):
        return 1
    if check_draw(board):
        return 0

    if is_maximizing:
        max_eval = -float('inf')
        for i in range(3):
            for j in range(3):
                if board[i][j] == ' ':
                    board[i][j] = 'O'
                    eval = minimax(board, depth + 1, False, alpha, beta)
                    board[i][j] = ' '
                    max_eval = max(max_eval, eval)
                    alpha = max(alpha, eval)
                    if beta <= alpha:
                        break
        return max_eval
    else:
        min_eval = float('inf')
        for i in range(3):
            for j in range(3):
                if board[i][j] == ' ':
                    board[i][j] = 'X'
                    eval = minimax(board, depth + 1, True, alpha, beta)
                    board[i][j] = ' '
                    min_eval = min(min_eval, eval)
                    beta = min(beta, eval)
                    if beta <= alpha:
                        break
        return min_eval

In [8]:
# Function to find the best move for the AI player ('O') based on the specified difficulty
def find_best_move(board, difficulty):
    if difficulty == EASY:
        # In easy mode, return a random move
        return random_move(board)
    elif difficulty == HARD:
        best_move = None
        best_eval = -float('inf')
        alpha = -float('inf')
        beta = float('inf')
        for i in range(3):
            for j in range(3):
                if board[i][j] == ' ':
                    board[i][j] = 'O'
                    eval = minimax(board, 0, False, alpha, beta)
                    board[i][j] = ' '
                    if eval > best_eval:
                        best_eval = eval
                        best_move = (i, j)
                    alpha = max(alpha, eval)
        # Return the best move determined by the minimax algorithm
        return best_move

In [9]:
# Function to get user input with validation
def get_user_input(prompt, valid_inputs):
    while True:
        try:
            user_input = input(prompt).lower()
            if user_input in valid_inputs:
                return user_input
            else:
                print("Invalid input. Please enter a valid option.")
        except KeyboardInterrupt:
            # Handle the KeyboardInterrupt (e.g., Ctrl+C) to exit gracefully
            exit()

In [10]:
# Function to display game statistics
def display_game_stats(stats):
    print("\nGame Statistics:")
    print(f"Player Wins: {stats['player_wins']}")
    print(f"AI Wins: {stats['ai_wins']}")
    print(f"Draws: {stats['draws']}\n")

In [11]:
# Function to undo the last 'num_moves' on the game board
def undo_last_moves(board, move_history, num_moves):
    if len(move_history) >= num_moves:
        # Loop 'num_moves' times to undo moves
        for _ in range(num_moves):
            row, col, player = move_history.pop()
            # Set the cell corresponding to the undone move to an empty space
            board[row][col] = ' '
        return True
    else:
        # Not enough moves in the history to undo
        return False

In [ ]:
# Function to manage the main game loop
def main():
    while True:
        current_player = 'X'  # Start with player 'X'

        print("Welcome to Tic-Tac-Toe!")
        ai_difficulty = get_user_input("Select AI difficulty (easy/hard): ", [EASY, HARD])

        reset_game()  # Reset the game state

        while True:
            display_board(board)  # Display the current state of the board

            if current_player == 'X':  # Human player's turn
                try:
                    move = input("Enter your move (row column): ")
                    if move.lower() == 'undo':
                        if undo_last_moves(board, move_history, 2):
                            continue
                        else:
                            print("Cannot undo further.")
                            continue
                    row, col = map(int, move.split())
                    if not (0 <= row < 3 and 0 <= col < 3) or board[row][col] != ' ':
                        print("Invalid move! Please enter a valid move.")
                        continue
                except ValueError:
                    print("Invalid input! Please enter row and column numbers.")
                    continue
            else:  # AI player's turn
                print("AI player is thinking...")
                row, col = find_best_move(board, ai_difficulty)

            move_history.append((row, col, current_player))
            board[row][col] = current_player  # Place the player's symbol on the board

            if check_winner(board, current_player):  # Check if the player has won
                display_board(board)
                print(f"{current_player} wins!")
                if current_player == 'X':
                    game_stats['player_wins'] += 1
                else:
                    game_stats['ai_wins'] += 1
                display_game_stats(game_stats)
                break
            elif check_draw(board):  # Check if the game is a draw
                display_board(board)
                print("It's a draw!")
                game_stats['draws'] += 1
                display_game_stats(game_stats)
                break

            current_player = 'O' if current_player == 'X' else 'X'  # Switch player for the next turn

        # Ask the player if they want to continue playing another game or exit
        play_again = get_user_input("Do you want to play another game? (yes/no): ", ["yes", "no"])
        if play_again == "no":
            break

# Start the game if this script is executed
if __name__ == "__main__":
    main()

Welcome to Tic-Tac-Toe!
Select AI difficulty (easy/hard): hard
 | | 
-----
 | | 
-----
 | | 
-----
Enter your move (row column): 1 1
 | | 
-----
 |X| 
-----
 | | 
-----
AI player is thinking...
O| | 
-----
 |X| 
-----
 | | 
-----
Enter your move (row column): 0 2
O| |X
-----
 |X| 
-----
 | | 
-----
AI player is thinking...
O| |X
-----
 |X| 
-----
O| | 
-----
Enter your move (row column): 1 0
O| |X
-----
X|X| 
-----
O| | 
-----
AI player is thinking...
O| |X
-----
X|X|O
-----
O| | 
-----
Enter your move (row column): 0 2
Invalid move! Please enter a valid move.
O| |X
-----
X|X|O
-----
O| | 
-----
Enter your move (row column): 0 1
O|X|X
-----
X|X|O
-----
O| | 
-----
AI player is thinking...
O|X|X
-----
X|X|O
-----
O|O| 
-----
